In [20]:
# import os
# import json
# import pprint
# import requests
# # import config 


#### download

In [21]:
# ''' Download path were all water level time series will be stored '''
# dir_download = 'data/altimetry-water-level/dahiti'
# if not os.path.isdir(dir_download):
# 	os.mkdir(dir_download)

# API_KEY = config.dahiti_key
# OUTPUT_FORMAT = 'netcdf' ## Option: ascci, json, netcdf, csv 

# url = 'https://dahiti.dgfi.tum.de/api/v2/list-targets/'
# args = {}
# ''' User configuration '''
# args['api_key'] = API_KEY
# ''' Search options '''
# # args['basin'] = 'Amazon'
# # args['continent'] = 'Asia'
# #args['country'] = 'de'
# args['min_lon'], args['max_lon'] = 100, 106   #### 66-106
# args['min_lat'], args['max_lat'] = 26, 45    #### 26-45


In [22]:
# ''' send request as method POST '''
# response = requests.post(url, data=args)
# if response.status_code == 200:
# 	''' convert json string in python list '''
# 	targets = json.loads(response.text)['data']
# 	print ('Dataset(s) found:',len(targets))		
# 	for target in targets:
# 		# print (target)		
# 		path_target_file = dir_download + '/' + str(target['dahiti_id']) + '.nc'
# 		if os.path.exists(path_target_file):
# 			print(f"File {target['dahiti_id']} already exists. Skipping download.")
# 			continue

# 		''' download water level time series '''
# 		url = 'https://dahiti.dgfi.tum.de/api/v2/download-water-level/'
# 		args = {}
# 		args['api_key'] = API_KEY
# 		args['dahiti_id'] = target['dahiti_id']
# 		args['format'] = OUTPUT_FORMAT		
# 		path_output = os.path.abspath(dir_download+'/'+str(target['dahiti_id'])+'.nc')
# 		print ('Downloading ... ',target['dahiti_id'],'->',target['target_name'].encode("utf8"),'('+path_output+')')		
# 		response_download = requests.post(url, json=args)
# 		if response_download.status_code == 200:
# 			with open(path_output, 'wb') as f:
# 				for chunk in response_download.iter_content(chunk_size=1024): 
# 					if chunk:
# 						f.write(chunk)
# 		else:
# 			print ('Error: `download-water-level` request failed!')
# 			data = json.loads(response_download.text)
# 			pprint.pprint(data)
# 			# sys.exit(0)
# else:
# 	print ('Error: `list-targets` request failed!')
# 	data = json.loads(response.text)
# 	pprint.pprint(data)



#### Preprocessing
Write the dahiti information into geopandas dataframe.

In [23]:
import netCDF4 as nc
from glob import glob
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs



In [24]:
dir_paths = 'data/water-height/dahiti'
path_lake = 'data/water-extent/lakes/HydroLakes_v10_hma_region.gpkg'
paths = glob(dir_paths+'/*.nc')
print(len(paths))


411


In [25]:
hma_lakes_gdf = gpd.read_file(path_lake)
hma_lakes_gdf.head(2)


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Vol_src,Depth_avg,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry
0,131,Alakol,Kazakhstan,Asia,SWBD,1,0,2919.34,567.40,2.96,...,1,22.1,55.947,13347.0,347,-1.0,63248.1,81.672917,46.152083,"POLYGON ((81.54844 46.50765, 81.55058 46.50721..."
1,136,Issyk Kul,Kyrgyzstan,Asia,SWBD,1,0,6195.93,608.84,2.18,...,1,280.5,72.678,276778.9,1601,-1.0,21917.0,77.427083,42.485417,"POLYGON ((78.21662 42.7446, 78.21667 42.7421, ..."


#### save the dahiti station information to geopandas dataframe.

In [26]:
ids, continents, contries, names, types, points = [], [], [], [], [], []
for path in paths:
    dset = nc.Dataset(path)
    ids.append(dset.dahiti_id)
    continents.append(dset.continent)
    contries.append(dset.country)
    names.append(dset.target_name.split(',')[0].strip())
    types.append(dset.target_name.split(',')[1].strip())
    points.append(Point(dset.longitude, dset.latitude))


In [27]:
dahiti_lake_station = {'dahiti_id': ids, 
     'continent': continents,
      'country': contries,
      'name': names,
      'type': types,
     'geometry': points}

In [28]:
hylak_id, hylak_geoms = [], []
for point in points:
    id_lake = hma_lakes_gdf.geometry.contains(point)    
    if not id_lake.any(): 
        hylak_geoms.append(None)
        hylak_id.append(None)
    else:
        hylak_geoms.append(hma_lakes_gdf.geometry[id_lake].values[0])
        hylak_id.append(hma_lakes_gdf['Hylak_id'][id_lake].values.item())


In [29]:
dahiti_lake_station.update({'Hylak_id': hylak_id, 'Hylak_geom': hylak_geoms})
dahiti_lake_station_gdf = gpd.GeoDataFrame(dahiti_lake_station, crs="EPSG:4326")
dahiti_lake_station_gdf.to_file('data/water-height/dahiti/dahiti_station_hma.gpkg', driver='GPKG')
dahiti_lake_station_gdf.head(2)


,dahiti_id,continent,country,name,type,geometry,Hylak_id,Hylak_geom
0,10782,Asia,China,Yarlung Tsangpo,River,POINT (91.715 29.2624),NaN,None
1,11126,Asia,China,Dulishi,Lake,POINT (81.9182 34.7704),14943.0,"POLYGON ((81.94249471028678 34.77582804361987,..."
